# Iata Code Download Exploration
 exploring how to scrape iata codes from the IATA website

In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output
from pathlib import Path

from time import sleep
from random import randint

base URL https://www.iata.org/en/publications/directories/code-search/?airport.page=1&airport.search=

prototyping parsing one page of the table

In [2]:
response = requests.get(
    url='https://www.iata.org/en/publications/directories/code-search/?airport.page=275&airport.search='
)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find_all('table', {'class': "datatable"})[1]
df = pd.read_html(str(table))
df = pd.concat(df)
df.head()

,City Name,3-letter city code,Location Name,3-letter location code
0,Carpentaria Downs,CFP,Carpentaria Downs,CFP
1,Carriacou,CRU,Carriacou,CRU
2,Carrizo Springs,CZT,Dimmit County,CZT
3,Carroll,CIN,Arthur N. Neu,CIN
4,Carson City,CSN,Carson,CSN


lets turn the datatable parser into its own function

In [5]:
def location_table_parser(html_response: str) -> pd.DataFrame:
    soup = BeautifulSoup(
        html_response,
        'html.parser'
    )
    table = soup.find_all(
        'table',
        {'class': "datatable"}
    )[-1]
    return pd.concat(
        pd.read_html(str(table))
    )

great that works! Next lets try parsing all the pages using a while loop and the sleep function

## Scraping all pages

this took waaay to long

In [6]:
dataframes = []
errored_pages = []
page = 1
#while True:
#    clear_output(wait=True)
#    print(f'Page {page}', end='')
#    url = (
#        f'https://www.iata.org/en/publications/directories/code-search/?airport.page={page}&airport.search='
#    )
#    response = requests.get(url)
#    try:
#        df = location_table_parser(response.text)
#        
#        if len(df) == 0:
#            break
#   except IndexError:
#        errored_pages.append(page)
#
#
#   dataframes.append(df)
#   if page%50 == 0:
#       data = pd.concat(dataframes)
#       data.to_csv('iata_codes.csv', index=False)
#
#    page += 1
#    print('...complete')
#    sleep(randint(2, 5))

#complete_dataset = pd.concat(dataframes)

lets try a different plan

## Scraping of codes that exists in the jet2 dataset

In [7]:
jet2_path = Path().cwd().parent.parent / 'data' / 'external' / 'jet2_data.json'
with open(jet2_path) as f:
    jet2 = json.load(f)
jet2 = pd.DataFrame(jet2['Data'])
jet2.head()

,country,label,code,isDepartureAirport,isDestinationAirport,destinationIataCodes,searchTerms,isEnabledForBooking,airportUrlKey
0,United Kingdom,Aberdeen,ABZ,True,False,,"[ABZ, Aberdeen, United Kingdom]",False,aberdeen
1,France,Albert (Picardie Somme),BYF,True,False,,"[BYF, Albert, (Picardie, Somme), France]",False,albert
2,Spain,Alicante ALC,ALC,True,True,BHX|EDI|NCL|MAN|GLA|BFS|STN|EMA|LBA,"[ALC, Alicante, ALC, Benidorm, Costa Blanca, S...",True,alicante
3,Spain,Almeria LEI,LEI,True,True,EMA|GLA|LBA|MAN|BHX|STN|NCL,"[LEI, Almeria, LEI, Spain]",True,almeria
4,Netherlands,Amsterdam AMS,AMS,True,True,LBA,"[AMS, Amsterdam, AMS, Holland, Netherlands, Ne...",True,amsterdam


In [8]:
codes = jet2['code'].unique()

In [13]:
dataframes = []
errors = []
for i, code in enumerate(codes):
    clear_output()
    print(f'{i +1 : d} of {len(codes)}')
    url = (
        f'https://www.iata.org/en/publications/directories/code-search/?airport.search={code}'
    )
    response = requests.get(url)
    for tries in range(2):
        if 'error' in BeautifulSoup(response.text).head:
            response = requests.get(url)
            sleep(2)
        else:
            break
    try:
        dataframes.append(
            location_table_parser(response.text)
        )
    except IndexError:
        errors.append(code)
    print(errors)
    sleep(randint(2, 5))

106 of 106
[]


In [14]:
code_data = pd.concat(dataframes)

In [15]:
errors

[]